# Introduction
Previously, I made an escher map to visualize central carbon metabolism of our strain. Here I will make the map from scratch. The only available maps are for amino acid degradation, not biosynthesis so will not be of much use to us. I will try to make the map so that each amino acid is coupled to a metabolite that is in the core-map we have generated previously.


In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

In [5]:
solution = model.optimize()

In [6]:
Builder(model=model, map_json = '../map/amino-acid-biosynthesis.json', reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 4.461457437274267, 'CAT': 0.05343860478337801, 'PDHam1hi': 0.0, 'CCP': 0.0, 'HY…

In making the map, I saw cysteine is produced in a strange way that I would not expect. Here I will make the changes needed to fix this. In our model, cysteine is made through the cgly_c metabolite. cgly is made from gthrd which is imported. 
Still I cannot remove the gthrd from the medium: we still do not get growht. So maybe this has to do with the cystein biosynthesis route. I should check it again here. 

In [8]:
model.metabolites.gthrd_c.name = 'Reduced glutathione'

LCYSTLY is the first setp of a reaction pathway that combines with AMBALY to give the same reaction as CYSTGL. Therefore I will remove the LCYSTLY and AMBALY reactions.
Same goes for LMETLY. 

In [9]:
model.remove_reactions(model.reactions.LCYSTLY)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [10]:
model.remove_reactions(model.reactions.LMETLY)

In [11]:
model.remove_reactions(model.reactions.AMBALY)

In [12]:
model.reactions.GLYOX.bounds = (-1000,0)

In [13]:
model.reactions.GTHPe.id = 'GTHPi'

In [14]:
model.reactions.GGTAe2.bounds = (0,1000)

In [15]:
model.reactions.GLYOX_1.id = 'LGTHL'

In [16]:
model.reactions.LGTHL.bounds = (-1000,0)

In [17]:
model.reactions.LALDPOR.id = 'LALDO2x'

In [18]:
model.reactions.LALDO2x.bounds = (-1000,0)

In [19]:
model.reactions.LALDO2x.add_metabolites({model.metabolites.nad_c:1, model.metabolites.nadh_c:-1, model.metabolites.h_c:1})

In [20]:
model.reactions.ACSERT.bounds = (0,1000)

I can't seem to figure out what the cause is of this strange gthrd pathway being used and why I cant remove it. I will now use the same approach as in notebook 20 to find out what biomass metabolite becomes blocked when i remove the gthrd exchange from the model. I would expect cysteine, as this is the direct was cysteine is being made. 

Turns out the cause is a combiantion of coa, succoa, accoa, gthrd, cysteine and methionine.

To fix the cysteine and methionine problem, suppling h2s solves the problem. So first I will look into how this is supplied and consumed.

In [21]:
rna = ['amp_c', 'gmp_c', 'cmp_c', 'ump_c']
dna = ['damp_c', 'dgmp_c', 'dcmp_c', 'dtmp_c']
purines = ['amp_c', 'gmp_c', 'damp_c', 'dgmp_c']
pyrimidines = ['cmp_c', 'ump_c', 'dcmp_c', 'dtmp_c']
energy = ['adp_c', 'atp_c']
aa = ['ser__L_c', 'phe__L_c', 'gly_c', 'ala__L_c', 'val__L_c', 'leu__L_c', 'ile__L_c', 'thr__L_c','tyr__L_c','trp__L_c','cys__L_c','met__L_c', 'lys__L_c', 'arg__L_c', 'his__L_c', 'asp__L_c', 'glu__L_c', 'asn__L_c', 'gln__L_c', 'pro__L_c']
cofactors = ['nad_c', 'nadh_c','nadph_c', 'nadp_c', 'fad_c', 'qh2_c']
other = ['ptrc_c', 'chor_c', 'thmpp_c', 'ribflv_c', 'fe3_c', 'fe2_c', 'cl_c', 'coa_c', 'succoa_c', 'accoa_c', 'gthrd_c']

In [22]:
other = ['', '', '', '', '', '', '', 'coa_c', 'succoa_c', 'accoa_c', 'gthrd_c']
aa = ['', '', '', '', '', '', '', '','','','cys__L_c','met__L_c', '', '', '', '', '', '', '', '']
with model:
    model.reactions.EX_gthrd_e.bounds = (0,0) #blocking exchange kills biomass
    model.add_boundary(model.metabolites.h2s_c, type = 'sink', reaction_id = 'test')
    for met in model.reactions.biomass.metabolites:
        if met.id in aa:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        if met.id in other:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        else: 
            continue
    solution = model.optimize()
    #print (model.metabolites.glu__D_c.summary())
    #print ('test flux:', solution['test'])
    #print ('test2 flux:', solution['test2'])
    
    print (solution.objective_value)

1.780908984682483


__H2S__
SO4 is added to the medium in real experiments, so we need to ensure this can properly be interconverted. It seems we are really missing some sulfur metabolism. Also we dont even have the sulfate metabolite. I will add those reactions in now.

In [23]:
model.reactions.SULR.bounds = (0,1000)

In [24]:
#sulfate metabolite and exchange 
model.add_metabolites(Metabolite(id='so4_e'))
model.add_metabolites(Metabolite(id='so4_c'))

In [25]:
model.metabolites.so4_c. name = 'Sulfate'
model.metabolites.so4_c.charge = -2
model.metabolites.so4_c.formula = 'O4S'
model.metabolites.so4_c.compartment = 'c'
model.metabolites.so4_c.annotation = model_e_coli.metabolites.so4_c.annotation

In [26]:
model.metabolites.so4_e. name = 'Sulfate'
model.metabolites.so4_e.charge = -2
model.metabolites.so4_e.formula = 'O4S'
model.metabolites.so4_e.compartment = 'e'
model.metabolites.so4_e.annotation = model_e_coli.metabolites.so4_e.annotation

In [27]:
model.add_boundary(model.metabolites.so4_e,type = 'exchange', reaction_id = 'EX_so4_e', lb = -1000, ub = 1000)

Reaction identifier,EX_so4_e
Name,Sulfate exchange
Memory address,0x028a91f391c8
Stoichiometry,so4_e <=> Sulfate <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [28]:
#add transport, proton symport
model.add_reaction(Reaction(id='SO4t'))

In [29]:
model.reactions.SO4t.name = 'Sulfate transport via proton symport'
model.reactions.SO4t.annotation['sbo'] = 'SBO:0000185'
model.reactions.SO4t.bounds = (0,1000)

In [30]:
model.reactions.SO4t.add_metabolites({
    model.metabolites.h_e:-1, 
    model.metabolites.so4_e:-1,
    model.metabolites.h_c:1, 
    model.metabolites.so4_c:1,
})

Next, I have to add the pathway encoded by EC 1.8.4.8, EC 2.7.1.25 and EC 2.7.7.4, as these are all annotated in the genome. They contain the assimilatory sulfate reduction pathway. 

In [31]:
#step 1: EC2.7.7.4
model.add_reaction(Reaction(id='SADT'))

In [32]:
#need to add the  Adenosine 5'-phosphosulfate metabolite first
model.add_metabolites(Metabolite(id='aps_c'))

In [33]:
model.metabolites.aps_c. name = 'Adenosine 5-phosphosulfate'
model.metabolites.aps_c.charge = -2
model.metabolites.aps_c.formula = 'C10H12N5O10PS'
model.metabolites.aps_c.compartment = 'c'
model.metabolites.aps_c.annotation = model_e_coli.metabolites.aps_c.annotation

In [34]:
model.reactions.SADT.name = 'Sulfate adenylyltransferase'
model.reactions.SADT.annotation = model_b_sub.reactions.SADT.annotation
model.reactions.SADT.bounds = (0,1000)

In [35]:
model.reactions.SADT.add_metabolites({
    model.metabolites.atp_c:-1, model.metabolites.h_c: -1, model.metabolites.so4_c:-1,
    model.metabolites.aps_c:1, model.metabolites.ppi_c: 1
})

In [36]:
#step 2 add EC 2.7.1.25 and paps metabolite

In [37]:
model.add_metabolites(Metabolite(id='paps_c'))

In [38]:
model.metabolites.paps_c. name = '3-Phosphoadenylyl sulfate'
model.metabolites.paps_c.charge = -4
model.metabolites.paps_c.formula = 'C10H11N5O13P2S'
model.metabolites.paps_c.compartment = 'c'
model.metabolites.paps_c.annotation = model_e_coli.metabolites.paps_c.annotation

In [39]:
model.add_reaction(Reaction(id='ADSK'))

In [40]:
model.reactions.ADSK.name = 'Adenylyl-sulfate kinase'
model.reactions.ADSK.annotation = model_e_coli.reactions.ADSK.annotation
model.reactions.ADSK.bounds = (0,1000)

In [41]:
model.reactions.ADSK.add_metabolites({
    model.metabolites.aps_c:-1, model.metabolites.atp_c:-1,
    model.metabolites.adp_c:1, model.metabolites.h_c:1, model.metabolites.paps_c:1
})

In [42]:
#step 3: add ec 1.8.4.8 and pap metabolite

In [43]:
model.add_metabolites(Metabolite(id='pap_c'))

In [44]:
model.metabolites.pap_c. name = 'Adenosine 3,5-bisphosphate'
model.metabolites.pap_c.charge = -4
model.metabolites.pap_c.formula = 'C10H11N5O10P2'
model.metabolites.pap_c.compartment = 'c'
model.metabolites.pap_c.annotation = model_e_coli.metabolites.pap_c.annotation

In [45]:
model.add_reaction(Reaction(id='PAPSR'))

In [46]:
model.reactions.PAPSR.name = 'Phosphoadenylyl-sulfate reductase (thioredoxin)'
model.reactions.PAPSR.annotation = model_e_coli.reactions.PAPSR.annotation
model.reactions.PAPSR.bounds = (0,1000)

In [47]:
model.reactions.PAPSR.add_metabolites({
    model.metabolites.paps_c:-1, model.metabolites.trdrd_c: -1, 
    model.metabolites.h_c: 2, model.metabolites.pap_c:1, model.metabolites.so3_c:1, model.metabolites.trdox_c:1
})

In [48]:
#need to add hydrolysis of pap to amp
model.add_reaction(Reaction(id='BPNT'))

In [49]:
model.reactions.BPNT.name = '3,5-bisphosphate nucleotidase'
model.reactions.BPNT.annotation = model_e_coli.reactions.BPNT.annotation
model.reactions.BPNT.bounds = (0,1000)

In [50]:
model.reactions.BPNT.add_metabolites({
    model.metabolites.h2o_c:-1, model.metabolites.pap_c:-1,
    model.metabolites.amp_c:1, model.metabolites.pi_c:1
})

Fixing the sulfate assimilation pathway solved the cysteine and methionine pathways when we remove the gthrd exchange. It also fixes the problem with the coa, succoa and accoa. The only blocked metabolite now is the gthrd metabolite. 

I need to look into its biosynthesis pathway to ensure this is possible and then the problem should be resolved.

In [51]:
#save & commit the above
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

# Gthrd biosynthesis
We've fixed the sulfate assimilation pathway, resolving the issues with cysteine and methionine. However the gthrd is still not biosynthesized as it should be. This is a problem and currentlly killing biomass, so needs to be fixed.

Only reaction that can produce gthrdis the LGTHL reaction. But this is in a cycle with the GLYOX reaction, just interconvertin gthrd with lgt__S_c. 

Generally, glutathione is produced and metabolized in the glutathione cycle. the GGTAe2 reaction in the model covers the breakdown of glutathione into glutamate. However the reactions converting the glutamte into y-hlutmyl-cysteine and further into glutathione are not present. I will check this in the genome if there is reason to beleive they should be present.

I can't find any reason to believe the complete cycle should be present. it seems that the Gamma-glutamyl transferase will likely only hydrolyse glutathione, due to some publications. (doi:10.1016/j.biochi.2010.01.021 and doi.org/10.1155/2014/216270). Also I do not really see why it is part of the biomass reaction, so this will be revisited after we look into that.

In [14]:
other = ['', '', '', '', '', '', '', '', '', '', 'gthrd_c']
with model:
    model.reactions.EX_gthrd_e.bounds = (0,0) #blocking exchange kills biomass
#     model.add_boundary(model.metabolites.h2s_c, type = 'sink', reaction_id = 'test')
    for met in model.reactions.biomass.metabolites:
#         if met.id in aa:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
        if met.id in other:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        else: 
            continue
    solution = model.optimize()
    #print (model.metabolites.glu__D_c.summary())
    #print ('test flux:', solution['test'])
    #print ('test2 flux:', solution['test2'])
    
    print (solution.objective_value)

1.6430375759493985


# Valine pathway
We expect valine to be produced via the two KARI reactions, that are incluced in the map currently. However, there is no flux through these reactions. here I will inspect why this is to be sure this is correct. Turns out this is because there is the KARA1 reaction too, which performs the same conversion, just without the inbetwen step of 3hmoa_c. As the net reaction is the same, and this could cause a cycle I will remove the two KARI reactions. This will also make the model more BiGG compliant which is useful for later purposes.

Then, I will adapt the reaction in the AA synthesis map. 

In [54]:
model.remove_reactions(model.reactions.KARI_23dhmb)

In [55]:
model.remove_reactions(model.reactions.KARI_3hmoa)

In [56]:
model.remove_metabolites(model.metabolites.get_by_id('3hmoa_c'))

In [57]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

# asn pathway
Also, asparagine is not being produced as one would expect. So here I will check how it is being synthesized curerntly to determine what should be fixed.

Asn is made via the ASNS2 reaction, which can assimilate ammonia. There is indication that the asparagine synthase enzyme in E. coli and B. subtilis can assimilate ammonium. Therefore I will leave this reaction in the model, and just adapt it in the map. 

In [52]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [53]:
model.reactions.biomass.metabolites

{<Metabolite h2o_c at 0x28a90008408>: -104.9974,
 <Metabolite qh2_c at 0x28a9001b9c8>: -0.029903042,
 <Metabolite coa_c at 0x28a900b8288>: -0.029903042,
 <Metabolite leu__L_c at 0x28a8ffdb048>: -0.36865919,
 <Metabolite ptrc_c at 0x28a8fff6488>: -0.029903042,
 <Metabolite his__L_c at 0x28a90003408>: -0.08256969,
 <Metabolite gmp_c at 0x28a942abcc8>: -0.145952,
 <Metabolite pro__L_c at 0x28a9429a608>: -0.17668818,
 <Metabolite asn__L_c at 0x28a942956c8>: -0.42971812,
 <Metabolite fad_c at 0x28a935d5f08>: -0.029903042,
 <Metabolite val__L_c at 0x28a935d5348>: -0.39907943,
 <Metabolite thr__L_c at 0x28a935cec88>: -0.2851532,
 <Metabolite phe__L_c at 0x28a935bdc88>: -0.15396106,
 <Metabolite succoa_c at 0x28a935ba608>: -0.029903042,
 <Metabolite fe2_c at 0x28a935bac48>: -0.029903042,
 <Metabolite fe3_c at 0x28a935c0688>: -0.029903042,
 <Metabolite amp_c at 0x28a9440a788>: -0.113187265,
 <Metabolite dcmp_c at 0x28a93acf508>: -0.023333636,
 <Metabolite atp_c at 0x28a93acfd48>: -105.027303042